In [ ]:
import geopy
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from google.colab import files

**Step 1.** Obtain dataset of swimmable (<110F) springs that includes spring temp, county name, latitude, longitude

In [ ]:
# Load thermal springs dataset
file = '/content/NCEI-thermal-springs.csv'
springs = pd.read_csv(file)

In [ ]:
# Get all string temperature rows 
H_rows = springs[springs['Max_surface_temp_F'] == 'H']
W_rows = springs[springs['Max_surface_temp_F'] == 'W']
B_rows = springs[springs['Max_surface_temp_F'] == 'B']

# Get only numeric temperature rows and convert to float
num_rows = springs[springs['Max_surface_temp_F'] != 'H'] 
num_rows = num_rows[num_rows['Max_surface_temp_F'] != 'W']
num_rows = num_rows[num_rows['Max_surface_temp_F'] != 'B']
num_rows = num_rows.drop([102, 1102]) # Random null rows
num_rows['Max_surface_temp_F'] = num_rows['Max_surface_temp_F'].astype(float)

# Convert W/H rows to integer values (W=25percentile of num_rows, H=50percentile) 
W_rows['Max_surface_temp_F'] = 79
H_rows['Max_surface_temp_F'] = 102

# Eliminate un-swimmable 
num_rows = num_rows[num_rows['Max_surface_temp_F'] <= 110]

# Gather all swimmable springs together
swimmers = pd.concat([num_rows, W_rows, H_rows], ignore_index=True)

# Remove Alaska and Hawaii
swimmers = swimmers[(swimmers['State'] != 'AK') & (swimmers['State'] != 'HI')]

# Drop unnecessary columns to leave only: Latitude, Longitude, Temp
swimmers.drop(columns=['State', 'Spring_name', 'Max_surface_temp_C', 'PP492', 'USGS_Circ_790', 'NOAA', 'AMS_map', 'USGS_quadrangle'], inplace=True)

# Reset index
swimmers = swimmers.reset_index(drop=True)

In [ ]:
# Get county names for springs
def get_location(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw

geolocator = geopy.Nominatim(user_agent='my-application')
locations = swimmers.apply(get_location, axis=1, geolocator=geolocator, lat_field='Latitude', lon_field='Longitude')

In [ ]:
# Get series of counties
locations_df = pd.DataFrame(locations.tolist())
locations_address = pd.DataFrame(locations_df['address'].tolist())
counties_only = locations_address['county']

# Add counties to swimmers df
swimmers['county'] = counties_only

In [ ]:
swimmers.head()

In [ ]:
# Save this dataframe as CSV so I don't have to go through the time-consuming geolocation step each time.
swimmers.to_csv('swimmers.csv')
files.download('swimmers.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Step 2.** Obtain census data by county that includes latitude/longitude of each county

In [ ]:
# Load swimmable thermal springs dataset
file = '/content/swimmers.csv'
swimmers = pd.read_csv(file)
swimmers.reset_index(inplace=True)
swimmers.drop(columns=['index', 'Unnamed: 0'], inplace=True)

In [ ]:
# Load census dataset
file = '/content/2017_census_county.csv'
census = pd.read_csv(file)

# Remove Alaska, Hawaii, Puerto Rico
census = census[(census['State'] != 'Alaska') & 
                (census['State'] != 'Hawaii') &
                (census['State'] != 'Puerto Rico')]

In [ ]:
# Load county name - lat/long data
file = '/content/county_lat_long.csv'
county_lat_long = pd.read_csv(file)

# Remove Alaska and Hawaii
county_lat_long = county_lat_long[(county_lat_long['State'] != 'AK') & 
                                  (county_lat_long['State'] != 'HI')]
# Rename FIPS column to CountyId
county_lat_long = county_lat_long.rename(columns={'FIPS': 'CountyId'})
# Drop unnecessary columns
county_lat_long.drop(columns=['State', 'County'], inplace=True)
county_lat_long

In [ ]:
# Add lat/long data to census dataframe
census = pd.merge(census, county_lat_long, how='inner', on=['CountyId'])
census = census.set_index('CountyId')

Step 3. Create target columns in census dataframe for (1) counties with and without swimmable springs, (2) numerical distance to closest spring, (3) categorical distance to closest spring

In [ ]:
# Target 1: True or False if a county has a swimmable spring

# Obtain list of counties with swimmable springs
county_list = swimmers['county'].tolist()
# Add column for whether a county has a spring or not
census['has_spring'] = census['County'].isin(county_list)

# Show percentages of targets (spring: 17.6%, No spring: 82.4%)
census['has_spring'].value_counts()

False    2562
True      546
Name: has_spring, dtype: int64

In [ ]:
# Target 2: Distance from county to nearest spring

# Create list of all spring coordinates
lat_list = swimmers['Latitude'].to_list()
lon_list = swimmers['Longitude'].to_list()
coordinates_zip = zip(lat_list, lon_list)
spring_coordinates = list(coordinates_zip)

# Create new census column with county lat-lon coordinates
census['coordinates'] = list(zip(census.Latitude, census.Longitude))

In [ ]:
# Function to calculate distance between two points on Earth
# Takes for two sets of (lat, lon), returns result in miles
def distance(county, spring):
    # Converts from degrees to radians.
    lon1 = radians(county[1])
    lon2 = radians(spring[1])
    lat1 = radians(county[0])
    lat2 = radians(spring[0])
    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in miles
    r = 3956
    # calculate the result
    return (c * r)

# Takes coordinates for 1 county and all springs (list), returns number of miles to closest spring in list
def find_min_distance(county_coordinates, spring_coordinates):
  min = 10000000
  for spring in spring_coordinates:
    dist = distance(county_coordinates, spring) 
    if dist < min:
      min = dist
  return round(min)

In [ ]:
# Get closest spring for each county (as measured from county centroid)
closest_springs = []
for row in range(len(census)):
  closest_springs.append(
      find_min_distance(census.iloc[row]['coordinates'], spring_coordinates))

census['closest_spring'] = closest_springs

194.82625482625483

In [ ]:
# Mean absolute error
from sklearn.metrics import mean_absolute_error
pred = census['closest_spring'].mean()
print('mean closest spring', pred)
error_total = 0
for i in range(len(census)):
  error_total += abs(census.iloc[i]['closest_spring'] - pred)
mae = error_total / (len(census))
print('mean absolute error', mae)

mean closest spring 194.82625482625483
mean absolute error 112.13938373011828


In [ ]:
# Target 3: Springs distance categories as <20mi, 20-50mi, >50mi
census['to_spring_cat'] = 0
census.loc[(census['closest_spring'] <= 20), 'to_spring_cat'] = '<20 mi'
census.loc[(census['closest_spring'] > 20) & (census['closest_spring'] <= 50), 'to_spring_cat'] = '20-50 mi'
census.loc[(census['closest_spring'] > 50), 'to_spring_cat'] = '>50 mi'

census['to_spring_cat'].value_counts(normalize=True)
# Baseline accuracy: 84% (always choosing >50mi)

>50 mi      0.846847
20-50 mi    0.089125
<20 mi      0.064028
Name: to_spring_cat, dtype: float64

In [ ]:
# Save final census dataframe as CSV to speed up future work.
census.to_csv('census.csv')
files.download('census.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>